# Compare Tycho and Aurora

In [ ]:
# %load imports.py
# %load ../imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as pltb
import matplotlib.pyplot as plt
import seaborn as sns
width=20
height=3
plt.rcParams["figure.figsize"] = (width,height)
sns.set(rc={'figure.figsize':(width,height)})

#import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import folium
import plotly.express as px
import plotly.graph_objects as go

import sys
import os

from sklearn.metrics import r2_score
import seaborn as sns

import statsmodels.api as sm

from d2e2f.visualization import visualize
import re

In [ ]:
loaded = catalog.load('trip_statistics_joined_thrusters')
df = pd.DataFrame()
for partition_id, loader in loaded.items():
    df_ = loader()
    df_['partition_id'] = partition_id
    df_['start_time'] = pd.to_datetime(df_['start_time'])
    df_['end_time'] = pd.to_datetime(df_['end_time'])
    
    df = df.append(df_, ignore_index=True)

df.sort_values(by='start_time', inplace=True)

In [ ]:
df.head()

In [ ]:
df['partition_id'].unique()

In [ ]:
df['ship'] = df['partition_id'].apply(lambda x:re.search(r'([^-]+)',x).groups(1)[0])

In [ ]:
fig = px.scatter(df, x='start_time',y='P', color='ship', width=1000, height=600, 
                color_discrete_sequence=['red','green'], hover_data=['trip_no','trip_time'])
fig.show()

In [ ]:
ships = df.groupby(by='ship')

In [ ]:
df_tycho=ships.get_group('tycho')
df_aurora=ships.get_group('aurora')


In [ ]:
df_tycho.info()

In [ ]:
df_tycho.head()

In [ ]:
str(df_tycho['start_time'].iloc[0])

In [ ]:
(df_tycho['start_time'].diff().iloc[1:] > "0").all()

In [ ]:
mask = df_tycho['start_time'].diff() > "1 days"

In [ ]:
mask = df_aurora['start_time'].apply(lambda x: ((x-df_tycho['start_time']).abs() < "0 days 01:00:00").any())
df_aurora = df_aurora.loc[mask].copy()

In [ ]:
df_cut = pd.concat([df_tycho, df_aurora])
df_cut.sort_values(by='start_time', inplace=True)
fig = px.scatter(df_cut, x='start_time',y='P', color='ship', width=1000, height=600, 
                color_discrete_sequence=['red','green'], hover_data=['trip_no','trip_time'])
fig.show()

In [ ]:
sns.displot(df_cut, x='P', hue='ship', kind="kde", bw_adjust=1, aspect=3)

In [ ]:
df_tycho.describe()

In [ ]:
df_aurora.describe()

In [ ]:
fig = px.scatter_3d(df_cut, x='start_time', y='sog', z='P', color='ship',
              opacity=0.5,
                    width=1000,
                    height=800
    )

fig.update_traces(marker=dict(size=2,
                         ))

fig.show()